In [2]:
# from google.colab import files
# uploaded = files.upload()
import numpy as np
import pandas as pd


import math
import json
# import numpy  as np
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
# from nltk.stem import WordNetLemmatizer
# wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = pd.read_csv('/content/drive/My Drive/NLP_project/train.csv')
data=data.fillna("unknown")
cmnt_text = list(data['comment_text'])
toxic = list(data['toxic'])
st = list(data['severe_toxic'])
ob = list(data['obscene'])
th = list(data['threat'])
ins = list(data['insult'])
ih = list(data['identity_hate'])
toxic_comments = []
severe_toxic_comments = []
obscene_comments = []
threat_comments = []
insult_comments = []
identity_hate_comments = []

for i in range(0,len(toxic)):
    if(toxic[i] == 1):
        toxic_comments+=[cmnt_text[i]]
        
for i in range(0,len(st)):
    if(st[i] == 1):
        severe_toxic_comments+=[cmnt_text[i]]
        
for i in range(0,len(ob)):
    if(ob[i] == 1):
        obscene_comments+=[cmnt_text[i]]
        
for i in range(0,len(th)):
    if(th[i] == 1):
        threat_comments+=[cmnt_text[i]]

for i in range(0,len(ins)):
    if(ins[i] == 1):
        insult_comments+=[cmnt_text[i]]
        
for i in range(0,len(ih)):
    if(ih[i] == 1):
        identity_hate_comments+=[cmnt_text[i]]
dict_comments = {"toxic":toxic_comments,"severe_toxic":severe_toxic_comments, "obscene":obscene_comments, "threat":threat_comments, "insult":insult_comments, "identity_hate":identity_hate_comments }


In [0]:
def list_to_sent(listis):
  listToStr = ' '.join([str(elem) for elem in listis]) 
  return listToStr

kis=list_to_sent(dict_comments['toxic'])

In [0]:
listofsent=[]
for cls,temp in dict_comments.items():
      listofsent.append(list_to_sent(temp))


In [0]:
# listofsent[0]

In [0]:
list_of_sent_for_cosine={}
freq_in_doc={} 
porter = PorterStemmer() 
word_freq_per_sent={} 
count=0
tk = RegexpTokenizer(r'[\w]+[-\']?[\w]*')
stop_words = set(stopwords.words('english'))

# for y,x in enumerate(list(data['comment_text'])):
# listofsent=[]
for y,p in enumerate(listofsent):
#     print(y,p,"\n")
    word_freq_per_sent[y]={}
    list_of_sent_for_cosine[y]=[]
    
#     tk = RegexpTokenizer(r'[\w]+[-\']?[\w]*')    
    newtokens=tk.tokenize(p)
    # print(newtokens)
#     stop_words = set(stopwords.words('english'))
    for w in newtokens:
#         w=w.lower()
        w=porter.stem(w.lower())
#         w=wordnet_lemmatizer.lemmatize(w.lower())
        if w not in stop_words:
            list_of_sent_for_cosine[y].append(w)
            if w in word_freq_per_sent[y]:
                word_freq_per_sent[y][w]+=1
            else:
                word_freq_per_sent[y][w]=1    
            if w in freq_in_doc:
                freq_in_doc[w].add(y)
            else:
                freq_in_doc[w]={y}
            # if w in Vocab:
            #     Vocab[w]+=1
            # else:
            #     Vocab[w]=1
for w_is in freq_in_doc:
    freq_in_doc[w_is]=len(freq_in_doc[w_is])

In [0]:
list_of_train_sent=list(list_of_sent_for_cosine.values())
list_of_train_sent_tags=list(list_of_sent_for_cosine.keys())
# list_of_train_sent

In [0]:
def cosine(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    return np.dot(v1, v2) / (np.sqrt(np.sum(v1**2)) * np.sqrt(np.sum(v2**2)))

In [0]:
def compute_tf(snt_word_count):
    tf={}
    len_of_doc=sum(snt_word_count.values())
#     print(sum(snt_word_count.values()))  
    for x,y in snt_word_count.items():
        tf[x]=1+math.log(1+y)#y/len_of_doc
    return tf  
    
def compute_idf(fr_in_doc,lenis):
    idf={}
#     print(fr_in_doc)
#     print(lenis)  
    for x,y in fr_in_doc.items():
#         print(x,y)
        idf[x]=math.log(lenis/(1+y))
    return idf      

In [0]:
term_freq_dict={}
for x,y in word_freq_per_sent.items():
#     print(y)
    term_freq_dict[x]=compute_tf(y)
# term_freq_dict    

In [0]:
invrs_doc_freq_dict=compute_idf(freq_in_doc,len(listofsent))

In [0]:
def compute_tf_idf(tf,idf,lenis):
#     print(tf,"\n\n\n====================")
#     print(idf)
    vector_list=[]
    tf_idf_dict={}
    for p,q in tf.items():
        if(p in idf.keys()):
#             print(p,q,idf[p])
#             print(p,tf[p],y)
            vector_list.append(q*idf[p])
            tf_idf_dict[p]=q*idf[p]
        else:
            vector_list.append(q*math.log(lenis))
            tf_idf_dict[p]=q*math.log(lenis)
    return vector_list,tf_idf_dict   


In [0]:
term_freq_invrs_doc_freq_dic={}
term_freq_invrs_doc_freq_dic_list={}
for indx,k in term_freq_dict.items():
    term_freq_invrs_doc_freq_dic_list[indx],term_freq_invrs_doc_freq_dic[indx]=compute_tf_idf(k,invrs_doc_freq_dict,len(listofsent))

#     print(k,"\n")

In [0]:
def prepocess(data):
#     tk = RegexpTokenizer(r'[\w]+[-\']?[\w]*')
#     stop_words = set(stopwords.words('english'))
    newtokens1=tk.tokenize(data)
#     print(newtokens1)
    #     stop_words = set(stopwords.words('english'))
#     sent_is=[]
    word_freq_per_sent_is={}
    for w1 in newtokens1:
#         w1=wordnet_lemmatizer.lemmatize(w1.lower())
        w1=porter.stem(w1.lower())
        if w1 not in stop_words:
            if w1 in word_freq_per_sent_is:
                word_freq_per_sent_is[w1]+=1
            else:
                word_freq_per_sent_is[w1]=1
#             sent_is.append(w1)
    return word_freq_per_sent_is 

In [0]:
def cosine(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)

    return np.dot(v1, v2) / (np.sqrt(np.sum(v1**2)) * np.sqrt(np.sum(v2**2)))

In [33]:
def per_query_sol(sentis,to_tf_idf_dict,idf_train,len_train_sent):   
    # print(sentis)
    option_preprocess=prepocess(sentis)
    dictis={}
    tf_of_option_preprocess=compute_tf(option_preprocess)
    # print(tf_of_option_preprocess)
    tf_idf_of_=compute_tf_idf(tf_of_option_preprocess,idf_train,len_train_sent)
    # print(tf_idf_of_)
    max_score_doc=0
    max_classis=""
    for x,y in to_tf_idf_dict.items():
      # print(x,y) 
      dictis[x]=0
      intersection_list=list(np.intersect1d(list(tf_of_option_preprocess.keys()),list(y.keys())))
      if(len(intersection_list)>0):
            # print(intersection_list)
            # print(len(intersection_list))
      # print("\n\n")
            new_tf_of_option_preprocess = { key: tf_of_option_preprocess[key] for key in intersection_list }
            new_y={ key1: y[key1] for key1 in intersection_list }
# #                 print(new_tf_of_option_preprocess,new_y)
            vec1_of_op,tf_idf_of_option_preprocess=compute_tf_idf(new_tf_of_option_preprocess,idf_train,len_train_sent)
            vec2_of_train_doc=list(new_y.values())
            # print(vec1_of_op,vec2_of_train_doc)
            cosine_sim=cosine(vec1_of_op,vec2_of_train_doc)
            dictis[x]=cosine_sim
            # print(cosine_sim)
      # print("\n\n\n")
# #                 print(tf_idf_of_option_preprocess,new_y)
            if(cosine_sim>max_score_doc):
                    max_score_doc=cosine_sim
                    max_classis=x
    return max_classis




test_data['comment_text'][0]="== Arabs are committing genocide in Iraq, but ..."
per_query_sol(test_data['comment_text'][0],term_freq_invrs_doc_freq_dic,invrs_doc_freq_dict,len(listofsent))

2

In [0]:
# classification_data['imdb_score']=classification_data.apply(lambda row:'D' if ( int(row.imdb_score)>=0 and int(row.imdb_score)<5) else ('C' if (int(row.imdb_score)>=5 and int(row.imdb_score)<6) else ('B' if (int(row.imdb_score)>=6 and int(row.imdb_score)<7) else ('A' if (int(row.imdb_score)>=7 and int(row.imdb_score)<8) else ('A++' if (int(row.imdb_score)>=8 and int(row.imdb_score)<=10) else 0) )))  , axis = 1)
  
# new_test_data['real_label']=new_test_data.apply(lambda row:0 if (row.toxic==1) else (1 if (row.severe_toxic==1) else (2 if (row.obscene==1) else (3 if (row.threat==1) else (4 if (row.insult==1) else (5 if (row.identity_hate==1) else 0)) )))  , axis = 1)
  
  


In [0]:
label_data = pd.read_csv('/content/drive/My Drive/NLP_project/test_labels.csv')
list_of_file_to_find_ac=[]
for x,y in enumerate(label_data['id']):
#         if(x<1000):
            if((label_data["toxic"][x] == 1 or label_data["severe_toxic"][x] == 1 or  label_data["obscene"][x] == 1 or label_data["threat"][x] == 1 or label_data["insult"][x] == 1 or label_data["identity_hate"][x] == 1 )):
                list_of_file_to_find_ac.append(y)

In [49]:
len(list_of_file_to_find_ac)

6243

In [0]:
# for k in list(test_data['comment_text']):
# dictIS=test_data
# len(test_data)
label_data = pd.read_csv('/content/drive/My Drive/NLP_project/test_labels.csv')
new_test_data=label_data
  
new_test_data['real_label']=new_test_data.apply(lambda row:0 if (row.toxic==1) else (1 if (row.severe_toxic==1) else (2 if (row.obscene==1) else (3 if (row.threat==1) else (4 if (row.insult==1) else (5 if (row.identity_hate==1) else 5000)) )))  , axis = 1)

new_test_data = new_test_data[new_test_data['real_label'] !=5000]

# list_of_file_to_find_ac=[]
# for x,y in enumerate(label_data['id']):
# #         if(x<1000):
#       # print(x,y)
#       # break

#             if((label_data["toxic"][x] == 1 or label_data["severe_toxic"][x] == 1 or  label_data["obscene"][x] == 1 or label_data["threat"][x] == 1 or label_data["insult"][x] == 1 or label_data["identity_hate"][x] == 1 )):
#                 list_of_file_to_find_ac.append(y)

In [0]:
test_datais= pd.read_csv('/content/drive/My Drive/NLP_project/test.csv')

new_test_data_withsent=pd.merge(test_datais, new_test_data, on='id')
new_test_data_withsent

for col in ["toxic","severe_toxic","threat","insult","identity_hate","obscene"]:
  if (col in new_test_data_withsent.columns):
      new_test_data_withsent=new_test_data_withsent.drop(columns=[col])
# new_test_data_withsent
# new_test_data_withsent

# per_query_sol(test_data['comment_text'][0],term_freq_invrs_doc_freq_dic,invrs_doc_freq_dict,len(listofsent))

In [0]:
# new_test_data_withsent['found_label']=new_test_data_withsent.apply(lambda row:per_query_sol(row.comment_text,term_freq_invrs_doc_freq_dic,invrs_doc_freq_dict,len(listofsent)))
# new_test_data_withsent
# len(new_test_data_withsent)
new_test_data_withsent.loc[:,'found_label'] = new_test_data_withsent.apply(lambda x: per_query_sol(x['comment_text'],term_freq_invrs_doc_freq_dic,invrs_doc_freq_dict,len(listofsent)), axis =1)

In [0]:
from sklearn.metrics import accuracy_score
# print("accuracy_score of tf-idf ",accuracy_score(list(new_test_data_withsent['real_label']),list(new_test_data_withsent['found_label'])))